In [1]:
import pandas as pd
import sqlite3
import os
import re
import glob

In [3]:
# ファイル名から都道府県・ホール名・日付を取得
def get_pref_hallName_date(csv_file):
    filename = os.path.basename(csv_file)
    match = re.match(r"(.+?)_(.+?)_(\d{4}-\d{2}-\d{2})\.csv", filename)
    if not match:
        raise ValueError("ファイル名は『都道府県_ホール名_YYYY-MM-DD.csv』の形式である必要があります")

    prefecture = match.group(1)
    hall_name = match.group(2)
    date = match.group(3)

    print(f"📄 ファイル名から抽出： {prefecture}, {hall_name}, {date}")
    
    return prefecture, hall_name, date

In [4]:
# 都道府県とホール名からIDを取得
def get_hall_id_from_db(conn, hall_name, prefecture_name=None):
    cursor = conn.cursor()
    
    if prefecture_name:
        # 都道府県名からIDを取得
        cursor.execute("SELECT prefecture_id FROM prefectures WHERE name = ?", (prefecture_name,))
        result = cursor.fetchone()
        if not result:
            return None
        prefecture_id = result[0]
        # 都道府県ありの場合のホール検索
        cursor.execute("""
            SELECT hall_id FROM halls
            WHERE name = ? AND prefecture_id = ?
        """, (hall_name, prefecture_id))
    else:
        # 都道府県が指定されない場合（曖昧検索注意）
        cursor.execute("SELECT hall_id, name FROM halls WHERE name LIKE ?", ('%' + hall_name + '%',))

    
    result = cursor.fetchone()
    return result[0] if result else None

In [5]:
# データベース接続
def csv_to_database(cursor, df, hall_id, date):
    for _, row in df.iterrows():
        model_name = row["machine_name"]
        unit_no = int(row["unit_no"])
        games = int(row["game"])
        medals = int(row["medals"])
        BB = int(row["BB"])
        RB = int(row["RB"])

    # 機種登録＆取得
        cursor.execute("INSERT OR IGNORE INTO models (name) VALUES (?)", (model_name,))
        cursor.execute("SELECT model_id FROM models WHERE name = ?", (model_name,))
        model_id = cursor.fetchone()[0]

    # 出玉データを登録（重複回避）
        cursor.execute("""
        INSERT OR IGNORE INTO results (
            hall_id, model_id, unit_no, date, game, BB, RB, medals
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (hall_id, model_id, unit_no, date, games, BB, RB, medals))
    
    print("✅ DataFrame の内容を results テーブルに登録しました。\n")


In [6]:
# --- 設定 ---
DB_PATH = "../anaslo_02.db"
csv_files = glob.glob("../csv/*.csv")

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

for csv_file in csv_files:
    # CSVファイル名から都道府県・ホール名・日付を取得
    prefecture, hall_name, date = get_pref_hallName_date(csv_file)
    df = pd.read_csv(csv_file)

    hall_id = get_hall_id_from_db(conn, hall_name, prefecture_name=prefecture)
    csv_to_database(cursor, df, hall_id, date)

conn.commit()
conn.close()

📄 ファイル名から抽出： 東京都, EXA FIRST, 2025-03-25
✅ DataFrame の内容を results テーブルに登録しました。

📄 ファイル名から抽出： 東京都, EXA FIRST, 2025-03-26
✅ DataFrame の内容を results テーブルに登録しました。

📄 ファイル名から抽出： 東京都, EXA FIRST, 2025-03-27
✅ DataFrame の内容を results テーブルに登録しました。

📄 ファイル名から抽出： 東京都, EXA FIRST, 2025-03-28
✅ DataFrame の内容を results テーブルに登録しました。

📄 ファイル名から抽出： 東京都, EXA FIRST, 2025-03-29
✅ DataFrame の内容を results テーブルに登録しました。

📄 ファイル名から抽出： 東京都, EXA FIRST, 2025-03-30
✅ DataFrame の内容を results テーブルに登録しました。

📄 ファイル名から抽出： 東京都, EXA FIRST, 2025-03-31
✅ DataFrame の内容を results テーブルに登録しました。

📄 ファイル名から抽出： 東京都, EXA FIRST, 2025-04-01
✅ DataFrame の内容を results テーブルに登録しました。

📄 ファイル名から抽出： 東京都, EXA FIRST, 2025-04-02
✅ DataFrame の内容を results テーブルに登録しました。

📄 ファイル名から抽出： 東京都, EXA FIRST, 2025-04-03
✅ DataFrame の内容を results テーブルに登録しました。

📄 ファイル名から抽出： 東京都, EXA FIRST, 2025-04-04
✅ DataFrame の内容を results テーブルに登録しました。

📄 ファイル名から抽出： 東京都, EXA FIRST, 2025-04-05
✅ DataFrame の内容を results テーブルに登録しました。

📄 ファイル名から抽出： 東京都, EXA FIRST, 2025-04-06
✅ DataFrame 